CYBOS PLUS 선택하고 상시 모의투자 계정으로 접속해야 한다. 

CYBOS Plus 도움말 파일을 참조하면 매수/매도 주문을 위한 클래스는 CpTrade 모듈의 CpTd0311입니다. 다만 주문과 관련된 객체를 사용하기에 앞서 먼저 기본적으로 필요한 초기화 과정을 수행해야 하는데, 이를 위해서는 CpTrade 모듈의 CpTdUtill 클래스를 사용해야 합니다.

그림 10.29는 CpTdUtil과 CpTd0311 클래스의 메서드를 호출하는 순서를 나타냅니다. 앞서 설명한 것처럼 CpTdUtil 클래스의 TradeInit 메서드를 호출해 먼저 초기화 과정을 수행해야 합니다. CpTdUtil.TradeInit() --> CpTd0311.SetInputValue() --> CpTd0311.BlockRequest()

초기화가 정상적으로 이뤄지면 CpTd0311 클래스를 사용하면 됩니다. 해당 클래스는 Request/Reply 타입이므로 먼저 매수/매도할 정보를 SetInputValue 메서드를 통해 설정한 후 BlockRequest 메서드를 호출해 매수/매도를 서버에 요청하면 됩니다.

CpTd0311은 Request/Reply 기반의 통신 방식을 사용하지만 GetDataValue 메서드는 사용할 수 없습니다. 따라서 매수/매도 주문에 대한 체결 내역은 GetDataValue 메서드를 통해 구하는 것이 아니라 CpDib 모듈의 CpConclusion 클래스를 통해 따로 구해와야 합니다.

본격적으로 API를 통해 가상계좌에서 매수를 수행해보겠습니다. 이를 위해 먼저 PyCharm과 같은 에디터를 관리자 모드로 실행해야 합니다. 참고로 매수/매도 API를 이용하려면 주식 시장의 운영 시간에 코드를 실행해야 합니다. 대신증권의 모의투자는 가상이긴 하지만 실제 시장의 운영 시간과 같은 시간에만 운영되기 때문입니다.

CpTrade 모듈의 CpTdUtil과 CpTd0311 클래스를 사용하기 위해 해당 클래스의 인스턴스를 먼저 생성합니다.


모의투자도 시간 잘 보고 해야한다. 근데 어차피 꼭 실제 매매를 하고 결과를 확인해야 하는 것은 아니기 때문에 크게 상관은 없다. GetDataValue()를 통해 직접 수익률 등을 계산할 수도 있다. 

## 2) 매수/매도하기

In [2]:
import win32com.client

In [3]:
instCpTdUtil = win32com.client.Dispatch("CpTrade.CpTdUtil")
instCpTd0311 = win32com.client.Dispatch("CpTrade.CpTd0311")

instCpTdUtil.TradeInit() # 주문을 위한 초기화: 0 뜨지만 연결 된 것. 

0

TradeInit 메서드를 통해 정상적으로 초기화가 수행되면 CpTd0311 클래스를 이용해 실제 주문을 수행하면 됩니다. 앞서 간단히 설명한 것처럼 CpTd0311 클래스는 Request/Reply 타입이므로 먼저 주문에 관련된 정보를 SetInputValue 메서드로 설정해야 합니다.

표 10.3은 CpTd0311 클래스에서 설정하는 주요 타입 및 입력 데이터의 종류를 나타낸 것입니다. 예를들어, 매수 주문을 넣고 싶다면 SetInputValue(0, 2)와 같은 방식으로 SetInputValue 메서드를 호출하면 됩니다.

모의투자 계좌를 통해 대신증권 종목 10주를 13,000원에 매수하는 주문을 넣어보겠습니다. 이를 위해 먼저 SetInputValue 메서드를 호출해 해당 주문 데이터를 설정해야 합니다. 주문에 필요한 계좌 번호는 CpTdUtil 클래스의 AccountNumber 속성을 통해 구할 수 있습니다. 계좌 번호가 여러 개인 경우 해당 속성은 리스트 형태로 구성되는데, 현재 모의 투자는 한 개의 계좌만 사용 중이므로 AccountNumber[0]을 통해 계좌를 구할 수 있습니다.

In [4]:
accountNumber = instCpTdUtil.AccountNumber[0]
instCpTd0311.SetInputValue(0, 2)
instCpTd0311.SetInputValue(1, accountNumber)

주문 종류 코드와 계좌 번호를 설정하고 나면 주문할 종목의 종목 코드를 설정합니다. 종목 코드는 타입 3에 설정하면 되며, 대신증권의 종목 코드는 'A003540'입니다.

타입 4와 타입 5에 주문 수량과 주문 단가를 입력하면 되는데, 이때 중요한 것은 4장의 표 4.2에서 설명한 호가 규칙에 맞게 주문해야 한다는 점입니다.

모든 설정이 완료되면 BlockRequest 메서드를 호출해 주문을 처리하도록 증권사 서버에 요청합니다.

In [7]:
instCpTd0311.SetInputValue(3, 'A003540') # 종목코드: 대신증권

instCpTd0311.SetInputValue(4, 10) # 주문수량: 10
instCpTd0311.SetInputValue(5, 13000) # 주문단가: 13000

In [8]:
instCpTd0311.BlockRequest() # 0 뜨지만 주문체결 확인 창도 뜨므로 잘 된 것 같다. 

0

위 코드를 통해 주식 매수 주문을 실행해도 전송한 주문이 잘 체결됐는지 아직은 확인할 수 없습니다. 앞에서 설명한 것처럼 주식 주문의 체결 내역은 CpConclusion 클래스를 통해 확인할 수 있는데, 해당 클래스는 지금까지 배운 Request/Reply 통신 방식의 클래스와 달리 사용하기가 어렵습니다.

따라서 당분간은 CYBOS(CYBOS Plus 아님)를 실행해 실시간으로 주문 내역이 잘 처리됐는지 확인하겠습니다. CYBOS Plus의 클래스 중 Request/Reply 방식이 아닌 Subscribe/Unsubscribe 방식의 클래스는 이 책의 후반부까지 모두 공부한 후 직접 도전해 보시기 바랍니다.